In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,Flatten,Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

In [8]:
base_dir, _ = os.path.splitext("../../Downloads/chest_xray")

In [9]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

train_normal = os.path.join(train_dir, 'NORMAL')
print ('Total training normal images:', len(os.listdir(train_normal)))
train_pneu = os.path.join(train_dir, 'PNEUMONIA')
print ('Total training pneu images:', len(os.listdir(train_pneu)))

val_normal = os.path.join(validation_dir, 'NORMAL')
print ('Total validation normal images:', len(os.listdir(val_normal)))
val_pneu = os.path.join(validation_dir, 'PNEUMONIA')
print ('Total validation pneu images:', len(os.listdir(val_pneu)))

test_dir = os.path.join(base_dir,'test')
test_normal = os.path.join(test_dir, 'NORMAL')
print ('Total test normal images:', len(os.listdir(test_normal)))
test_pneu = os.path.join(test_dir, 'PNEUMONIA')
print ('Total test pneu images:', len(os.listdir(test_pneu)))

Total training normal images: 1341
Total training pneu images: 3875
Total validation normal images: 8
Total validation pneu images: 8
Total test normal images: 234
Total test pneu images: 390


In [10]:
weight_norm = (1 / len(os.listdir(train_normal)))*(len(os.listdir(train_normal))+len(os.listdir(train_pneu)))/2.0 
weight_pneu = (1 / len(os.listdir(train_pneu)))*(len(os.listdir(train_normal))+len(os.listdir(train_pneu)))/2.0
class_weight = {0: weight_norm, 1: weight_pneu}
class_weight

{0: 1.9448173005219984, 1: 0.6730322580645162}

In [11]:
image_size = 224 # All images will be resized to 160x160
batch_size = 64

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary',
                shuffle=False)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

print("Number of layers in the base model: ", len(base_model.layers))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Number of layers in the base model:  155


In [ ]:
base_model.trainable = True

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 120

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dropout(.3),
  keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())


model2.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(.3))


model2.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))

model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.3))
model2.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model2.compile(optimizer = SGD(learning_rate = .001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 50
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size
# lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
es = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
cb_list = [es]
history2 = model2.fit(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=test_generator,
                              validation_steps=validation_steps,
                              callbacks = cb_list )

In [ ]:
predictions = []
for x in model2.predict(test_generator):
    for z in x:
        predictions.append(np.round(z))
        
metrics.confusion_matrix(test_generator.classes,predictions)

Run with weights

Visualizing

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

image_batch, label_batch = next(iter(train_ds))

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        if label_batch[n]:
            plt.title("PNEUMONIA")
        else:
            plt.title("NORMAL")
        plt.axis("off")

In [ ]:
show_batch(image_batch.numpy(), label_batch.numpy())

-Rename model
-create performance visualizations
-explain layers of the model

In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(16, kernel_size = (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(168, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [14]:
model.compile(optimizer = 'adam',
              loss='binary_crossentropy',
              metrics=['accuracy',keras.metrics.Recall(name='recall')])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
epochs = 25
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size
history = model.fit(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=test_generator,
                              validation_steps=validation_steps,
                              class_weight=class_weight)

Epoch 1/25
81/81 [==============================] - 116s 1s/step - loss: 2.8577e-04 - accuracy: 1.0000 - recall: 0.9939 - val_loss: 8.8749e-07 - val_accuracy: 0.7222 - val_recall: 0.9939
Epoch 2/25
81/81 [==============================] - 116s 1s/step - loss: 4.0637e-04 - accuracy: 0.9998 - recall: 0.9939 - val_loss: 1.1605e-04 - val_accuracy: 0.7304 - val_recall: 0.9940
Epoch 3/25
81/81 [==============================] - 119s 1s/step - loss: 4.3559e-05 - accuracy: 1.0000 - recall: 0.9940 - val_loss: 0.0789 - val_accuracy: 0.7304 - val_recall: 0.9940
Epoch 4/25
81/81 [==============================] - 123s 2s/step - loss: 2.1760e-05 - accuracy: 1.0000 - recall: 0.9941 - val_loss: 3.5115e-04 - val_accuracy: 0.7357 - val_recall: 0.9941
Epoch 5/25
81/81 [==============================] - 126s 2s/step - loss: 3.1674e-05 - accuracy: 1.0000 - recall: 0.9941 - val_loss: 1.7742e-07 - val_accuracy: 0.7446 - val_recall: 0.9942
Epoch 6/25
81/81 [==============================] - 126s 2s/step - lo

In [19]:
model.save('pneu_model2.h5')

In [ ]:
#checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("pneu_model.h5",
#                                                    save_best_only=True)

In [ ]:
#callbacks=[checkpoint_cb